In [ ]:
# Data Extraction

In [1]:
import pandas as pd                     # importing for reading the file 
from bs4 import BeautifulSoup           # importing for data extraction
from tqdm import tqdm                   # importing tqdm for processing
import multiprocessing                  # importing for multiprocessing the data 
import requests                         # importing for http protocols or interactions with webserver
import chardet                          # importing for knowing the encoding type of the text file 
import re                               # importing for removing uneccessary character in the text such as hyphens commas, fullstops
import os                               # importing for setting file location

import nltk                                                           # importing nltk library
import nltk.data                                                      # going through data module  
from nltk.tokenize import word_tokenize,pos_tag                       # importing the word tokenizer 
from nltk.corpus import cmudict                        
from nltk.corpus import stopwords
tokenizer = nltk.data.load('tokenizers/punkt/PY3/english.pickle')     # loading the english pickle of words 

In [2]:
df = pd.read_excel('Input.xlsx')        # reading the file 
df.head()                               # head of the file

,URL_ID,URL
0,123.0,https://insights.blackcoffer.com/rise-of-telem...
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...
4,432.0,https://insights.blackcoffer.com/rise-of-telem...


In [122]:
def collect_data(start,end):                                                       # function to data extraction
    for i in tqdm(df.values[start:end]):                                           # looping
        res      = requests.get(i[1])                                              # requesting for access the web
        soup     = BeautifulSoup(res.content,'html.parser')                        # parsing through the content
        try:                                                      
            heading  = soup.title.text                                             # getting the title
            if len(soup.find('div',class_ = 'td-post-content tagdiv-type').find_all('p')) > 10:  # check the len of p
                contents = ''.join([t.text for t in soup.find('div',class_ = 'td-post-content tagdiv-type').find_all('p')]) # feeding to content variable
            else:                                                                  # else
                contents = soup2.find('div',class_ = 'td-post-content tagdiv-type').find('p').text + soup2.find('div',class_ = 'td-post-content tagdiv-type').find('ul').text # combining the ul tags and paragraph tags together to the content
        except:
            heading  = soup.title.text                                             # heading for exception 
            for j in range(len(soup.find_all('div',class_ = 'tdb-block-inner td-fix-index'))):  # loop through length of div class
                if soup.find_all('div',class_ = 'tdb-block-inner td-fix-index')[j].find_all('p') != []: # check if the val not empty
                    content = ''.join([t.text for t in soup.find_all('div',class_ = 'tdb-block-inner td-fix-index')[j].find_all('p')]) # feed to the content 
        
        with open('Collections/'+str(i[0])+'.txt','w') as fd:                      # opening a folder called collection and writing the data onto the file 
            fd.write(heading)                                                      # headers collected
            fd.write('\n')                                                         # spacing 
            fd.write(contents)                                                     # contents collected 
            fd.close()                                                             # closing the file


In [123]:
p1 = multiprocessing.Process(target = collect_data,args =(0,57))           # multiprocessing from 0 to 57 to the function
p2 = multiprocessing.Process(target = collect_data,args =(57,114))         # multiprocessing from 57 to 114 to the function
p1.start()                                                                 
p2.start()                                                                 
p1.join()
p2.join()

100%|███████████████████████████████████████████| 57/57 [02:12<00:00,  2.33s/it]


In [3]:
with open('MasterDictionary/positive-words.txt','r') as fd2:            # opening file of positive words 
    text = fd2.read()                                                   # reading the file 
    fd2.close()                                                         # closing the file

In [4]:
with open('MasterDictionary/negative-words.txt', 'rb') as file:          # opening file of negative words as in readbin
    result = chardet.detect(file.read())                                 # using chardet library to check the encoding of the file 
    
with open('MasterDictionary/negative-words.txt', 'r', encoding=result['encoding']) as fd3: # opening againg the file of negative words 
    ntext = fd3.read()                                                                     # reading it     
    fd3.close()                                                                            # closing it 

In [5]:
####################################################################################################################
###########################1.2.Creating a dictionary of Positive and Negative words#################################
####################################################################################################################

master_dict = {}                          # creating master dict

positive = []                             # empty positive list 
negative = []                             # empty negative list
for pos in text.split('\n'):              # loop through positive text
    positive.append(pos)                  # loading the value to positive list
for neg in ntext.split('\n'):             # loop through negative text
    negative.append(neg)                  # loading the value to negative list
    
master_dict['positive'] = positive        # loading positive list to master dict
master_dict['negative'] = negative        # loading negative list to master dict 

In [105]:
def text_analysis(start,stop):
    def count_cleaned_words(text):
        cleaned_word_count = len(words)
        return cleaned_word_count

    def count_syllables(word):
        word = word.lower()
        if word.endswith("es") or word.endswith("ed"):
            return max(1, len(re.findall(r'[aeiouy]+', word)))
        else:
            return len(re.findall(r'[aeiouy]+', word))

    def count_syllables_in_text(text):
        words = re.findall(r'\b\w+\b', text)                                   # Tokenize the text into words
        syllable_counts = [count_syllables(word) for word in words]
        return syllable_counts

    def find_complex_words(text):
        words = re.findall(r'\b\w+\b', text)                                   # Tokenize the text into words
        complex_words = [word for word in words if count_syllables(word) > 2]
        return complex_words

    # counting the total number of personal pronoun present in the paragraph or the data extracted
    def count_pp(para):
        personal_pronouns = ['I','we','you','he','she','it','they','my','mine','our','ours','your','yours','his','her',
                         'hers','its','their','theirs','myself','yourself','himself','herself','itself','ourselves',
                         'yourselves','themselves']
        ls = []
        for i in para.split(' '):
            if i in personal_pronouns:
                ls.append(i)
        return len(ls)
    
    def calculate_average_word_length(text):
        words = text.split()
        total_char_count = sum(len(word) for word in words)
        total_word_count = len(words)

        if total_word_count == 0:
            return 0  # Avoid division by zero

        average_word_length = total_char_count / total_word_count
        return average_word_length


    
    
    
    
    data = []
    for i in tqdm(df.values[start:stop]):                                                   # going through the values   
        with open('Collections/' + str(i[0]) + '.txt','r') as fd:         # opening the text file 
            text = fd.read()                                              # reading the text file
        # cleaning the text by removing the stopwords
        custom_stopwords = set()
        for filename in os.listdir(stopwords_folder):
            if filename.endswith(".txt"):
                with open(os.path.join(stopwords_folder, filename), "r",encoding = result['encoding']) as file:
                    custom_stopwords.update(line.strip() for line in file)

        words          = word_tokenize(text)                                              # Tokenize the paragraph into words
        filtered_words = [word for word in words if word.lower() not in custom_stopwords] # Remove stopwords
        clean_para     = " ".join(filtered_words)                                         # Join the filtered words to reconstruct the paragraph
        
    #####################################################################################################################
    ######################################1.3. Extracting Derived variables##############################################
    #####################################################################################################################
        tot_sent = len(tokenizer.tokenize(clean_para))                          # calculating the total number of sentences in the article 
        sent = tokenizer.tokenize(clean_para)                                   # sentences in the file 
        word_pattern = r"[\w'+,-]+"                                             # word pattern 
        words = re.findall(word_pattern, clean_para)


        # positive scoring 
        p = 0
        n = 0
        for words in text.split(' '):
            if (words) in master_dict['positive']:
                p += 1
            elif (words) in master_dict['negative']:
                n += 1
        polarity_score = round((p-n)/((p+n)+0.000001),3)
        subjective_score = round(((p + n)/((len(words))+0.000001)),3)
        complex_words = len(find_complex_words(clean_para))
        
    #####################################################################################################################
    ##########################################2. Analysis of Readability#################################################
    #####################################################################################################################

        average_sentence_length = round(len(words) / tot_sent,3)
        try:
            Per_Complex_words       = round((complex_words / len(words)*100),3)
        except:
            Per_Complex_words       = 0
        fog_idx                 = round(0.4 * (average_sentence_length + Per_Complex_words),3)

    #####################################################################################################################
    #####################################3.Average Number of Words Per Sentence##########################################
    #####################################################################################################################

        average_word_length = round(len(words) / tot_sent,3)         # number of words per 

    #####################################################################################################################
    ##########################################4.complex Word count#######################################################
    #####################################################################################################################

        complex_words

    #####################################################################################################################
    ###############################################5. Word count#########################################################
    #####################################################################################################################

        tot_word = count_cleaned_words(clean_para)

    #####################################################################################################################
    ##########################################6. Syllable Count Per Word#################################################
    #####################################################################################################################

        total_syllable = sum(count_syllables_in_text(text))

    #####################################################################################################################
    ###########################################7. personal pronoun#######################################################
    #####################################################################################################################
        
        count_personal_pro = count_pp(text)
        
    #####################################################################################################################
    ###########################################8.average word length#####################################################
    #####################################################################################################################
               
        average_length = round(calculate_average_word_length(text),3)
            
    #####################################################################################################################
        
        data.append([i[0],i[1],p,n,polarity_score,subjective_score,average_sentence_length,Per_Complex_words,fog_idx,average_word_length,complex_words,tot_word,total_syllable,count_personal_pro,average_length])
    final = pd.DataFrame(data,
                         columns= ['URL_ID','URL','POSITIVE SCORE','NEGATIVE SCORE','POLARITY SCORE',
                                   'SUBJECTIVITY SCORE','AVG SENTENCE LENGTH','PERCENTAGE OF COMPLEX WORDS',
                                   'FOG INDEX','AVG NUMBER OF WORDS PER SENTENCE','COMPLEX WORD COUNT','WORD COUNT',
                                   'SYLLABLE PER WORD','PERSONAL PRONOUNS','AVG WORD LENGTH'])    
    
    
    return final

In [107]:
output = text_analysis(0,114)


100%|█████████████████████████████████████████| 114/114 [00:11<00:00,  9.85it/s]


In [108]:
output.to_excel('Final.xlsx',index = False)